In [1]:
import pandas as pd
from rdkit import Chem
import sys
sys.path.insert(0, "/users/yhb18174/SimDMTA/scripts/dataset/")
from new_dataset_fns import DatasetFormatter

In [2]:
df = pd.read_csv("/users/yhb18174/SimDMTA/datasets/test_dataset/ChEMBL_docking_all.csv", index_col='ID')
df = df.drop(columns=["Unnamed: 0"])


In [3]:
smiles_only_df = df[['SMILES']]
smiles_only_df


,SMILES
ID,
CHEMBL3979343,Cc1ccc2c(c1)C(=O)N(c3cnc(nc3N2C)Nc4cccc(c4)S(=...
CHEMBL4089152,CCn1c2c(c(=O)[nH]c1=O)C(C3=C([NH+](CC3=N2)CC=C...
CHEMBL4084046,CC(=O)C(C(c1ccc(cc1)Cl)c2ccc(cc2)Cl)C(=O)C
CHEMBL4069010,CC(=O)CC(c1ccccc1)c2c(c3c(cccn3)oc2=O)[O-]
CHEMBL4071717,CC(=O)N(C)c1ccc(cc1)NC(=O)COc2ccc(cc2Cl)Br
...,...
CHEMBL4069308,c1ccc(cc1)Cc2[nH]c3c(c4c(s3)CCCCC4)c(=O)n2
CHEMBL1507539,c1ccc2c(c1)c(nn(c2=O)CC3COC4(O3)CCCCC4)O
CHEMBL188678,c1ccc2c(c1)c3cccc(c3s2)c4cccc5c4oc(cc5=O)N6CCOCC6


In [4]:
formatter = DatasetFormatter()

In [5]:
preprocessed_mols = formatter._processMols(df=df)
preprocessed_mols

[15:27:01] Can't kekulize mol.  Unkekulized atoms: 1 2 3
[15:27:01] Can't kekulize mol.  Unkekulized atoms: 1 2 3
[15:27:05] Tautomer enumeration stopped at 176 tautomers: max transforms reached
[15:27:06] Tautomer enumeration stopped at 183 tautomers: max transforms reached


,SMILES,CNN_affinity,Affinity(kcal/mol),Mol,Canon_SMILES,Kekulised_SMILES,pH_adjusted_Canon_SMILES,Lilly_rules_pass,Lilly_rules_warning,Lilly_rules_SMILES,oe_logp
ID,,,,,,,,,,,
CHEMBL3979343,Cc1ccc2c(c1)C(=O)N(c3cnc(nc3N2C)Nc4cccc(c4)S(=...,7.472,-8.97,<rdkit.Chem.rdchem.Mol object at 0x154ff3ae1970>,C=CC(=O)N1CCN(S(=O)(=O)c2cccc(Nc3ncc4c(n3)N(C)...,C=CC(=O)N1CCN(S(=O)(=O)C2=CC(NC3=NC=C4C(=N3)N(...,Cc1ccc2c(c1)C(=O)N(c3cnc(nc3N2C)Nc4cccc(c4)S(=...,False,(1 matches to 'michael_rejected'),C=CC(=O)N1CCN(S(=O)(=O)C2=CC(=CC=C2)NC2=NC=C3C...,1.166000
CHEMBL4089152,CCn1c2c(c(=O)[nH]c1=O)C(C3=C([NH+](CC3=N2)CC=C...,6.817,-7.76,<rdkit.Chem.rdchem.Mol object at 0x154ff3aae270>,C=CC[NH+]1CC2=Nc3c(c(=O)[nH]c(=O)n3CC)C(c3ccc(...,C=CC[NH+]1CC2=NC3=C(C(=O)NC(=O)N3CC)C(C3=CC=C(...,CCn1c2c(c(=O)[nH]c1=O)C(C3C(=N2)CN(C3=O)CC=C)c...,True,D(95) enol_or_vinyl_ether:enamine_2:too_many_a...,C=CCN1CC2=NC3=C(C(=O)NC(=O)N3CC)C(C3=CC=C(C)C=...,1.152000
CHEMBL4084046,CC(=O)C(C(c1ccc(cc1)Cl)c2ccc(cc2)Cl)C(=O)C,6.220,-7.11,<rdkit.Chem.rdchem.Mol object at 0x154ff3aae040>,CC(=O)C(C(C)=O)C(c1ccc(Cl)cc1)c1ccc(Cl)cc1,CC(=O)C(C(C)=O)C(C1=CC=C(Cl)C=C1)C1=CC=C(Cl)C=C1,CC(=O)C(C(c1ccc(cc1)Cl)c2ccc(cc2)Cl)C(=O)C,True,None,CC(=O)C(C(C)=O)C(C1=CC=C(Cl)C=C1)C1=CC=C(Cl)C=C1,4.653000
CHEMBL4069010,CC(=O)CC(c1ccccc1)c2c(c3c(cccn3)oc2=O)[O-],5.935,-7.28,<rdkit.Chem.rdchem.Mol object at 0x154ff3aae120>,CC(=O)CC(c1ccccc1)c1c([O-])c2ncccc2oc1=O,CC(=O)CC(C1=CC=CC=C1)C1=C([O-])C2=C(C=CC=N2)OC1=O,CC(=O)CC(c1ccccc1)c2c(c3c(cccn3)oc2=O)[O-],True,None,CC(=O)CC(C1=CC=CC=C1)C1=C(O)C2=C(C=CC=N2)OC1=O,2.164001
CHEMBL4071717,CC(=O)N(C)c1ccc(cc1)NC(=O)COc2ccc(cc2Cl)Br,6.121,-7.48,<rdkit.Chem.rdchem.Mol object at 0x154ff3aae190>,CC(=O)N(C)c1ccc(NC(=O)COc2ccc(Br)cc2Cl)cc1,CC(=O)N(C)C1=CC=C(NC(=O)COC2=CC=C(Br)C=C2Cl)C=C1,CC(=O)N(C)c1ccc(cc1)NC(=O)COc2ccc(cc2Cl)Br,True,D(84) aniline_acyl_newd:bromine,CC(=O)N(C)C1=CC=C(NC(=O)COC2=CC=C(Br)C=C2Cl)C=C1,3.552000
...,...,...,...,...,...,...,...,...,...,...,...
CHEMBL4069308,c1ccc(cc1)Cc2[nH]c3c(c4c(s3)CCCCC4)c(=O)n2,5.850,-8.13,<rdkit.Chem.rdchem.Mol object at 0x154ff3a66200>,O=c1[nH]c(Cc2ccccc2)nc2sc3c(c12)CCCCC3,O=C1N=C(CC2=CC=CC=C2)NC2=C1C1=C(CCCCC1)S2,c1ccc(cc1)Cc2[nH]c(=O)c3c4c(sc3n2)CCCCC4,True,None,O=C1N=C(CC2=CC=CC=C2)NC2=C1C1=C(CCCCC1)S2,4.137000
CHEMBL1507539,c1ccc2c(c1)c(nn(c2=O)CC3COC4(O3)CCCCC4)O,5.835,-8.03,<rdkit.Chem.rdchem.Mol object at 0x154ff3a66270>,O=c1[nH]n(CC2COC3(CCCCC3)O2)c(=O)c2ccccc12,O=C1C2=CC=CC=C2C(O)=NN1CC1COC2(CCCCC2)O1,c1ccc2c(c1)c(=O)[nH]n(c2=O)CC3COC4(O3)CCCCC4,True,D(50) acetal_both_in_ring:hydrazide_cyclic,O=C1C2=CC=CC=C2C(=O)NN1CC1COC2(CCCCC2)O1,2.237000
CHEMBL188678,c1ccc2c(c1)c3cccc(c3s2)c4cccc5c4oc(cc5=O)N6CCOCC6,6.809,-9.65,<rdkit.Chem.rdchem.Mol object at 0x154ff3a662e0>,O=c1cc(N2CCOCC2)oc2c(-c3cccc4c3sc3ccccc34)cccc12,O=C1C=C(N2CCOCC2)OC2=C1C=CC=C2C1=C2SC3=CC=CC=C...,c1ccc2c(c1)c3cccc(c3s2)c4cccc5c4oc(cc5=O)N6CCOCC6,True,D(33) too_many_atoms,O=C1C=C(N2CCOCC2)OC2=C1C=CC=C2C1=C2SC3=CC=CC=C...,5.121000


In [8]:
desc_df, full_df = formatter._calculateDescriptors(df=preprocessed_mols)

In [ ]:
formatter._makeDockingCSV()